### progress report for 07/08/2016



# My setup

I've opened a server on AWS (amazon) that supports GPU based on ami-b36981d8. I've only need to update torch on the machine and clone neural-style from github.

I've used a stft/istf to create style/content spectrugrams and send them to AWS with ssh2/scp, run neural-style algorithm and return the transfered image back to my PC for reconstruction of the wav. This allows be to check several algorithm parameters and wav inputs quickly without a need for human intervention in-betweeners. The code to run the "full-loop" setup is matlab/full_loop.m.

# Experiments

I've tried style transfer from several contents to several styles.
For the stft I've used 8kHz sampling rate with 512 freq (positive) bins, 32mSec frames with 28mSec. For upto 2 Sec audio segments this gives no more then 512 time frames, so the maximal image is 512x512.

Tried these inputs in various configurations

* voice male (3 words)
* voice female (1 word)
* chirp (from 100Hz to 3000Hz)
* dtmf tones (12 diferent tones)
* noise



# Example results

Here is an example of interesting transfer was seen from chirp sound to male voice content

### Voice in style of Chirp

<img src="../jpgs/style_Chirp_sq_na_100_3000VsContent_Ben_holechParams__Weight1000_Imsize513_Iters500_InitImage_Original_color0.fig.jpg">

Below are the original audios and the transfered one

In [24]:
from IPython.display import Audio
### Style
Audio(url="audio/chirp_sq_na_100_3000.wav")

In [25]:
### content
Audio(url="audio/ben_holech.wav")

In [27]:
### transfered
Audio(url="audio/res/style_Chirp_sq_na_100_3000VsContent_Ben_holechParams__Weight1000_Imsize513_Iters500_InitImage_Original_color0_time.fig.wav")

# Other interesting results gallery
<img src="../jpgs/style_DtmfVsContent_Chirp_sq_na_100_3000Params__Weight2000_Imsize513_Iters250_InitImage_Original_color0.fig.jpg">

<img src="../jpgs/style_DtmfVsContent_Ben_holechParams__Weight10000_Imsize513_Iters200_InitImage_Original_color0.fig.jpg">

<img src="../jpgs/style_DtmfVsContent_Ben_holechParams__Weight1000_Imsize513_Iters100_InitImage_Original_color0.fig.jpg">

<img src="../jpgs/style_ChirpVsContent_Ben_holechParams__Weight10000_Imsize513_Iters100_InitImage_Original_color0.fig.jpg">

<img src="../jpgs/style_Chirp_sq_na_100_3000VsContent_NoiseParams__Weight1000_Imsize513_Iters100_InitImage_Original_color0.fig.jpg">

<img src="../jpgs/style_Chirp_sq_na_100_3000VsContent_DtmfParams__Weight2000_Imsize513_Iters250_InitImage_Original_color0.fig.jpg">

<img src="../jpgs/style_Ben_holechVsContent_DtmfParams__Weight10000_Imsize513_Iters200_InitImage_Original_color0.fig.jpg">

<img src="../jpgs/style_Ben_holechVsContent_NoiseParams__Weight10000_Imsize513_Iters200_InitImage_Original_color0.fig.jpg">


